# Setup & Data Import

## Setup Global Configs and Import Libraries

In [ ]:
# SETUP & IMPORT
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pathlib import Path
from scipy import stats
import itertools
import re
import missingno as msno

# Display options for clarity
pd.set_option('display.width', 160)
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', lambda v: f"{v:,.4f}")

# Set visual themes
sns.set_theme(style='whitegrid', context='notebook', font_scale=1.05)
plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'

In [ ]:
# --- Define Paths ---
salary_path = Path('Salary_Data.csv')
survey_path = Path('survey_respondents_info.csv')
superstore_path = Path('Superstore 2023.csv')

# --- Validate File Existence ---
for path in [salary_path, survey_path, superstore_path]:
    if not path.exists():
        raise FileNotFoundError(f"❌ File not found: {path.resolve()}")

# --- Load with Delimiter Fallback ---
def load_csv_safely(path):
    try:
        return pd.read_csv(path, sep=',', engine='python')
    except Exception:
        print(f"⚠️ Failed with ',' — retrying with ';' for {path.name}...")
        return pd.read_csv(path, sep=';', engine='python')

# --- Load Both Datasets ---
salary_df = load_csv_safely(salary_path)
survey_df = load_csv_safely(survey_path)
superstore_df = load_csv_safely(superstore_path)

print(f"✅ Salary_Data loaded: {salary_df.shape[0]:,} rows × {salary_df.shape[1]} columns")
print(salary_df.info())
display(salary_df.head())

print(f"✅ Survey_Respondents_Info loaded: {survey_df.shape[0]:,} rows × {survey_df.shape[1]} columns")
print(survey_df.info())
display(survey_df.head())

print(f"✅ Superstore_Data loaded: {superstore_df.shape[0]:,} rows × {superstore_df.shape[1]} columns")
print(superstore_df.info())
display(superstore_df.head())

## Define Necessary Functions

In [ ]:
# Function to split CamelCase while keeping acronyms intact
def split_camel_case_keep_acronyms(name):
    """
    Convert CamelCase to Snake_Case while keeping acronyms intact.
    
    Examples:
    - 'MostFavourite' -> 'Most_Favourite'
    - 'CustomerMPIValue' -> 'Customer_MPI_Value'
    - 'NPSScore' -> 'NPS_Score'
    """
    # Insert underscore between:
    #   1. a lowercase letter and uppercase letter (e.g., tM -> t_M)
    #   2. but NOT between consecutive uppercase letters (e.g., MPI stays MPI)
    return re.sub(r'(?<=[a-z])(?=[A-Z])', '_', name)

In [ ]:
# Create a function to check ID uniqueness
def check_id_uniqueness(df, id_column='id'):
    """
    Check if the specified ID column in a DataFrame contains unique values.
    Prints duplicate IDs and their counts if any are found.
    """

    """
    Steps:
    1. Count frequency of each ID.
    2. If all frequencies are 1, IDs are unique.
    3. If any frequency > 1, there are duplicate IDs.
    4. Print duplicate IDs with their counts.
    """
    # Check if the ID column exists
    if id_column not in df.columns:
        print(f"❌ Column '{id_column}' not found in DataFrame, try a correct column name.")
        return

    # Step 1: Count frequency of each ID
    id_counts = df[id_column].value_counts()

    # Step 2: Check if all IDs are unique
    if (id_counts == 1).all():
        print("✅ All IDs are unique.")
    else:
        print("❌ Duplicate IDs found!")

        # Step 3: Filter IDs where frequency > 1
        duplicates = id_counts[id_counts > 1]

        # Step 4: Print duplicate IDs and their counts
        print("\nDuplicated IDs and their counts:")
        print(duplicates)

## Change Columns Formatting

In [ ]:
# Check column names
dfs = [
    salary_df,
    survey_df,
    superstore_df
]

for df in dfs:
    print(list(df.columns))

In [ ]:
# Replace # with _ in column names
for df in dfs:
    df.columns = df.columns.str.replace('#', '_', regex=False)
    print(list(df.columns))

In [ ]:
# Split CamelCase while preserving acronyms
for df in dfs:
    df.columns = [split_camel_case_keep_acronyms(col) for col in df.columns]
    print(list(df.columns))

In [ ]:
# Replace ' ' with _ in column names
for df in dfs:
    df.columns = df.columns.str.replace(' ', '_', regex=False)
    print(list(df.columns))

In [ ]:
# Convert all column names to lowercase
for df in dfs:
    df.columns = df.columns.str.lower()
    print(list(df.columns))

In [ ]:
# Check column names after formatting
for df in dfs:
    print(list(df.columns))

# Dataframe 1

## Select Dataset and Handle Grammar Errors

In [ ]:
# Select dataset to analyze
current_df = superstore_df.copy()

In [ ]:
# Print column names
print("Data columns:", list(current_df.columns))

In [ ]:
# Check unique values to identity grammar and data issues
for column_name in current_df.columns:
    print(f"Unique values in column '{column_name}':")
    print(current_df[column_name].unique())
    print("\n")

In [ ]:
# Fix grammar issues manually if any
replacement_dict = {}           # Add replacements as needed
target_col = []                 # Only apply one column at a time

# Dictionary is not blank, so apply replacements
if len(replacement_dict) == 0 or len(target_col) == 0:
    print("ℹ️ replacement_dict and target_col are empty — no replacements applied.")
else:
    for col in target_col:
        if col in current_df.columns:
            current_df[col] = current_df[col].replace(replacement_dict)
            print("✅ Replacements applied successfully to column:", col)
            print(f"Unique values in '{col}' after replacements:", current_df[col].unique())

## Define Columns

In [ ]:
# Define columns
categorical_cols = ['ship_mode', 'segment', 'country/region', 'city', 'state', 'region', 'category', 'sub-category'] # Change respectively to your dataset
numerical_cols = ['sales', 'quantity', 'discount', 'profit']   # Change respectively to your dataset
date_cols = ['order_date', 'ship_date']  # Add date columns if any
data_cols = current_df.columns.to_list()

# Validate presence of expected columns
missing_expected = [c for c in categorical_cols + numerical_cols if c not in current_df.columns]
if missing_expected:
    print("⚠️ Warning: These expected columns are missing in the dataset:", missing_expected)

# Coerce categorical columns
for col in categorical_cols:
    if col in current_df.columns:
        current_df[col] = current_df[col].astype('category')

# Attempt to convert numerical columns robustly
for col in numerical_cols:
    if col in current_df.columns:
        # Strip common formatting issues then coerce
        current_df[col] = (current_df[col]
                   .astype(str)
                   .str.replace('$', '', regex=False)
                   .str.replace(',', '', regex=False)
                   .str.strip())
        current_df[col] = pd.to_numeric(current_df[col])

# Convert date columns
for col in date_cols:
    if col in current_df.columns:
        # Strip common formatting issues then parse dates
        current_df[col] = (current_df[col]
                     .astype(str)
                     .str.replace('/', '-', regex=False)
                     .str.replace('.', '-', regex=False)
                     .str.replace('\\', '-', regex=False)
                     # Trim timestamp if present
                     .str.split(' ').str[0]
                     .str.strip())
        current_df[col] = pd.to_datetime(current_df[col], format='%d-%m-%Y').dt.strftime('%m/%d/%Y')

print("\nDataFrame info():")
print(current_df.info())

display(current_df.head())

## Data Quality Assessment

In [ ]:
# Check uniqueness of 'id' column 
check_id_uniqueness(current_df, id_column='row_id')

In [ ]:
# Descriptive statistics (numeric)
print("\nDescriptive statistics (numeric):")
display(current_df.describe().round(2))

# Check duplicate rows
duplicate_rows = int(current_df.duplicated().sum())
print(f"Duplicate rows: {duplicate_rows:,}")

In [ ]:
# Missingness summary
miss_summary = (
    current_df.isna()
    .sum()
    .rename("Number of Missing")
    .to_frame()
)
miss_summary["Percentage of Missing"] = (miss_summary["Number of Missing"] / len(current_df) * 100).round(2)
miss_summary = miss_summary.rename_axis("Column")
print("Summary Table of Missing Values:")
display(miss_summary.sort_values("Number of Missing", ascending=False))

# Summarize missing count
msno.bar(current_df, figsize=(16, 5), color='steelblue')
plt.title("Missing Values by Column")
plt.show()

# Find missing pattern
msno.heatmap(current_df, figsize=(8, 4))
plt.title("Correlation of Missingness Between Columns")
plt.show()

In [ ]:
# Find missing pattern
target_col = []
group_col = []
for col in group_col:
    na_rate = current_df.groupby(col)[target_col].apply(lambda x: x.isna().mean())
    #print("\n")
    #print(na_rate)

    plt.figure(figsize=(min(16, 2 + 0.5*na_rate.shape[0]), 0.5*na_rate.shape[0] + 2))
    sns.heatmap(na_rate, annot=True, fmt=".1%", cmap="YlOrRd", cbar=True)
    plt.title(f"Missing-rate heatmap by `{col}`")
    plt.xlabel("Target numeric columns")
    plt.ylabel(col)
    plt.tight_layout()
    plt.show()

## Handle Missing Columns

In [ ]:
# Median imputation for numerical columns by group
target_col = []             # Apply one column at a time
group_col = []              # Grouping columns to calculate median
imputation_method = 'mean'  # Options: 'mode', 'mean', 'median'

if len(target_col) == 0 or len(group_col) == 0:
    print("ℹ️ target_col and group_col are empty — no replacements applied.")
else:
    for col in target_col:
        if col not in current_df.columns:
            print(f"❌ Column '{col}' not found — skipping.")
        else:
            # Select imputation method
            if imputation_method == 'mode':
                imputed_series = current_df.groupby(group_col, dropna=True, observed=True)[col].transform(lambda s: s.mode().iloc[0] if not s.mode().empty else np.nan)
            elif imputation_method == 'mean':
                imputed_series = current_df.groupby(group_col, dropna=True, observed=True)[col].transform('mean')
            elif imputation_method == 'median':
                imputed_series = current_df.groupby(group_col, dropna=True, observed=True)[col].transform('median')
            else:
                print("⚠️ Invalid imputation_method. Choose from 'mode', 'mean', or 'median'.")
                break

            # --- Create new imputed column ---
            out_col = f"{col}_imputed"
            current_df[out_col] = current_df[col].fillna(imputed_series)

            # --- Report results ---
            before = int(current_df[col].isna().sum())
            after  = int(current_df[out_col].isna().sum())
            print(f"✅ '{col}' imputed using {imputation_method} per group ({', '.join(group_col)}).")
            print(f"   Missing before: {before}, after: {after}")
                

In [ ]:
# Overwrite original column with imputed values if needed
for col in target_col:
    out_col = f"{col}_imputed"
    if out_col in current_df.columns:
        current_df[col] = current_df[out_col]
        current_df.drop(columns=[out_col], inplace=True)
        print(f"✅ Overwrote '{col}' with imputed values and dropped '{out_col}'.")

## Outlier Detection
Outliers are identified by IQR because IQR method is robust to outliers. In real world, data is commonly non-normal and skewed, making z-score methods less precise in detecting outliers.
This cell analyzes `current_df` (DataFrame 1) for:
- Outlier detection using both IQR (1.5*IQR)
- Visual diagnostics: boxplots + pairwise scatter vs target-like numeric columns.

In [ ]:
# OUTLIER DETECTION
# Calculate IQR for numerical columns
Q1 = current_df[numerical_cols].quantile(0.25)
Q3 = current_df[numerical_cols].quantile(0.75)
IQR = Q3 - Q1

# Identify outliers using IQR
outlier_condition = (current_df[numerical_cols] < (Q1 - 1.5 * IQR)) | (current_df[numerical_cols] > (Q3 + 1.5 * IQR))
outliers_iqr = (outlier_condition).sum()
outliers_iqr = outliers_iqr.rename("IQR Outliers").to_frame().sort_values(by="IQR Outliers", ascending=False)
print("Number of outliers detected by IQR:")
print(outliers_iqr)
print('-'*50)

# Outlier marking columns if outliers exists
for col in numerical_cols:
    if outliers_iqr.loc[col, "IQR Outliers"] > 0:
        current_df[f'{col}_outlier'] = outlier_condition[col]

# Visualize distributions with boxplots

plt.figure(figsize=(min(16, 4*len(numerical_cols)), 4*len(numerical_cols)))
for i, col in enumerate(numerical_cols, 1):

    # Capitalize column names for display
    col_display = str.capitalize(col)
    
    plt.subplot(len(numerical_cols), 3, i)
    sns.boxplot(x=current_df[col], color='steelblue')
    plt.title(f'Boxplot of {col_display}')
plt.tight_layout()
plt.show()

## Univariate Analysis

In [ ]:
# Bar chart for categorical columns (show 15 most frequent categories)

# Exclude columns if needed
exclude_cat = ['country/region']  
used_cat = [col for col in categorical_cols if col not in exclude_cat]

# Set up plot grid
cols = 2
rows = int(np.ceil(len(used_cat) / cols))
plt.figure(figsize=(14, 4 * rows))

for i, col in enumerate(used_cat, 1):
    plt.subplot(rows, cols, i)
    ax = sns.countplot(
        data=current_df,
        y=col,
        order=current_df[col].value_counts().index[:15]
    )

    # --- Add data labels ---
    for container in ax.containers:
        ax.bar_label(container, fmt='%d', label_type='edge', fontsize=9, padding=2)

    plt.title(f'Count of {col}', fontsize=12)
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.xticks(rotation=45)

plt.tight_layout()
plt.show()



## Bivariate Analysis

In [ ]:
# 1. Correlation Matrix of numerical columns
from pandas.api.types import CategoricalDtype

# 1) Define ORDERS (lowest → highest)
ship_mode_order = [
    "Standard Class",
    "Same Day",
    "Second Class",
    "First Class",
]

segment_order = [
    "Consumer",
    "Home Office",
    "Corporate",
]

category_order = [
    "Furniture",
    "Office Supplies",
    "Technology",
]

# 2) Encode ordinals → integer scores (NaN if unknown/missing)
df_enc = current_df.copy()

ord_specs = {
    "ship_mode": ship_mode_order,
    "segment": segment_order,
    "category": category_order,
}

encoded_cols = []
for col, order in ord_specs.items():
    # set ordered categorical
    ctype = CategoricalDtype(categories=order, ordered=True)
    df_enc[col] = df_enc[col].astype(str).where(df_enc[col].notna(), np.nan)  # preserve NaN
    df_enc[col] = df_enc[col].astype(ctype)
    # create numeric code column (keep NaN where category missing)
    new_col = f"{col}_ord"
    codes = df_enc[col].cat.codes.replace(-1, np.nan)
    df_enc[new_col] = codes
    encoded_cols.append(new_col)

# 3) Build list of numeric columns (original numerics + encoded ordinals)
num_cols_final = list(df_enc.select_dtypes(include=[np.number]).columns)

# If you only want your existing numerical_cols + the encoded ordinals:
num_cols_final = [c for c in numerical_cols if c in df_enc.columns] + encoded_cols

# 4) Spearman correlation (handles monotonic ordinal relationships)
corr = df_enc[num_cols_final].corr(method="spearman")

# 5) Plot heatmap (upper triangle)
mask = np.triu(np.ones_like(corr, dtype=bool))
plt.figure(figsize=(10, 12))
sns.heatmap(corr, mask=mask, annot=True, cmap="coolwarm", fmt=".2f", square=True,
            cbar_kws={"shrink": .8}, linewidths=.5)
plt.title("Correlation Matrix: Numerical + Encoded Ordinals (Spearman)")
plt.tight_layout()
plt.show()


In [ ]:
# 2. Scatter plots for pairwise numerical columns

# Generate all unique pairs of numerical columns
pairs = list(itertools.combinations(numerical_cols, 2))

# Determine grid layout automatically
cols = 2
rows = int(np.ceil(len(pairs) / cols))

plt.figure(figsize=(12, 4 * rows))

for i, (x_col, y_col) in enumerate(pairs, 1):
    # Capitalize column names for display
    x_display = str.capitalize(x_col)
    y_display = str.capitalize(y_col)
    plt.subplot(rows, cols, i)
    sns.scatterplot(data=current_df, x=x_col, y=y_col, color='steelblue', edgecolor='k', alpha=0.7)
    plt.title(f'{y_display} vs {x_display}')
    plt.xlabel(x_display)
    plt.ylabel(y_display)

plt.tight_layout()
plt.show()



In [ ]:
# 3. Create box plots for categorical columns vs. numerical columns

# Exclude columns if needed
exclude_cat = ['country/region','city']  
exclude_num = ['discount']
used_cat = [col for col in categorical_cols if col not in exclude_cat]
used_num = [col for col in numerical_cols if col not in exclude_num]

# Set up plot grid
cols = 2                # number of boxplots per row
row_height = 4          # adjust height for spacing

for cat_col in used_cat:
    rows = int(np.ceil(len(used_num) / cols))
    plt.figure(figsize=(14, row_height * rows))

    for j, num_col in enumerate(used_num, 1):
        plt.subplot(rows, cols, j)

        cat_display = str.capitalize(cat_col)
        num_display = str.capitalize(num_col)

        order = current_df[cat_col].value_counts().index[:10]  # limit categories

        sns.boxplot(
            data=current_df,
            y=cat_col,
            x=num_col,
            order=order
        )

        plt.title(f'{num_display}', fontsize=11, fontweight='bold')
        plt.xlabel(cat_display)
        plt.ylabel(num_display)
        plt.xticks(rotation=45)

    plt.suptitle(f'Boxplots grouped by {cat_col}', y=1.02, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

In [ ]:
# 3. Create bar plots for categorical columns vs. sum of numerical columns
# Exclude columns if needed
exclude_cat = ['country/region','city']  
exclude_num = []
used_cat = [col for col in categorical_cols if col not in exclude_cat]
used_num = [col for col in numerical_cols if col not in exclude_num]

# Set up plot grid
cols = 2                # number of bar plots per row
row_height = 4          # adjust height for spacing

for cat_col in used_cat:
    rows = int(np.ceil(len(used_num) / cols))
    plt.figure(figsize=(14, row_height * rows))

    for j, num_col in enumerate(used_num, 1):
        plt.subplot(rows, cols, j)

        cat_display = str.capitalize(cat_col)
        num_display = str.capitalize(num_col)

        order = current_df[cat_col].value_counts().index[:10]  # limit categories

        ax = sns.barplot(
            data=current_df,
            y=cat_col,
            x=num_col,
            order=order,
            estimator=np.nansum,
            errorbar=None
        )

        # --- Add data labels ---
        for container in ax.containers:
            ax.bar_label(container, fmt='%d', label_type='edge', fontsize=9, padding=2)

        plt.title(f'{num_display}', fontsize=11, fontweight='bold')
        plt.xlabel(f'Sum of {num_display}') # x is the total
        plt.ylabel(num_display)
        plt.xticks(rotation=0)

    plt.suptitle(f'Barplots grouped by {cat_col}', y=1.02, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

In [ ]:
# 1. Create a pair plot of the numerical columns
# Exclude columns if needed
exclude_cat = ['country/region','city','state']  
exclude_num = ['discount']
used_cat = [col for col in categorical_cols if col not in exclude_cat]
used_num = [col for col in numerical_cols if col not in exclude_num]

# Create pairplot
sns.pairplot(current_df[used_num], diag_kind='kde', markers='o')
plt.suptitle('Pair Plot of Numerical Columns', y=1.02)
plt.show()

# 2. Create a pair plot of numerical columns colored by a categorical column
for cat_col in used_cat:
    if current_df[cat_col].nunique() > 10:
        print(f"⚠️ Categorical column '{cat_col}' has more than 10 unique values, skipping pair plot.")
        continue
    sns.pairplot(current_df, vars=used_num, hue=cat_col, diag_kind='kde', markers='o')
    plt.suptitle(f'Pair Plot of Numerical Columns colored by {cat_col}', y=1.02)
    plt.show()

## Export Data

In [ ]:
#Drop redundant columns containing outlier flags if any
outlier_flag_cols = [col for col in current_df.columns if 'outlier' in col.lower()]
for col in outlier_flag_cols:
    current_df.drop(columns=[col], inplace=True)
    print(f"✅ Dropped outlier flag column: {col}")
    
# Overwrite original dataframe with current dataframe
superstore_df = current_df.copy()
print("✅ Overwrote 'superstore_df' with current dataframe.")


# Export Data
superstore_df.to_csv('superstore_data_cleaned.csv', index=False)
